In [7]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()
model = ChatGroq(model="llama-3.3-70b-versatile",temperature=0.1)   

In [8]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [9]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content

    return {'joke': response}

In [10]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

In [11]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [12]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty.',
 'explanation': 'A classic play on words. This joke is funny because it uses a pun to create a clever connection between the setup and the punchline. \n\nThe joke starts by asking why the pizza is in a bad mood, which sets up the expectation that the reason will be something related to emotions or a personal issue. However, the punchline subverts this expectation by using a wordplay on "crusty."\n\nIn this context, "crusty" has a double meaning. In one sense, the crust is a part of a pizza, referring to the outer layer of the bread. But "crusty" can also mean being irritable, gruff, or having a bad temper. So, when the joke says the pizza is "feeling a little crusty," it\'s making a humorous connection between the pizza\'s crust (the bread part) and the emotional state of being crusty (irritable). \n\nThe humor comes from the unexpected twist on the word\'s meaning, creating a cle

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty.', 'explanation': 'A classic play on words. This joke is funny because it uses a pun to create a clever connection between the setup and the punchline. \n\nThe joke starts by asking why the pizza is in a bad mood, which sets up the expectation that the reason will be something related to emotions or a personal issue. However, the punchline subverts this expectation by using a wordplay on "crusty."\n\nIn this context, "crusty" has a double meaning. In one sense, the crust is a part of a pizza, referring to the outer layer of the bread. But "crusty" can also mean being irritable, gruff, or having a bad temper. So, when the joke says the pizza is "feeling a little crusty," it\'s making a humorous connection between the pizza\'s crust (the bread part) and the emotional state of being crusty (irritable). \n\nThe humor comes from the unexpected twist on the word\'s mean

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty.', 'explanation': 'A classic play on words. This joke is funny because it uses a pun to create a clever connection between the setup and the punchline. \n\nThe joke starts by asking why the pizza is in a bad mood, which sets up the expectation that the reason will be something related to emotions or a personal issue. However, the punchline subverts this expectation by using a wordplay on "crusty."\n\nIn this context, "crusty" has a double meaning. In one sense, the crust is a part of a pizza, referring to the outer layer of the bread. But "crusty" can also mean being irritable, gruff, or having a bad temper. So, when the joke says the pizza is "feeling a little crusty," it\'s making a humorous connection between the pizza\'s crust (the bread part) and the emotional state of being crusty (irritable). \n\nThe humor comes from the unexpected twist on the word\'s mea

In [15]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.',
 'explanation': 'A clever play on words. This joke is funny because it uses a common phrase associated with romantic relationships, "tangled up," and gives it a literal twist. In this case, the spaghetti is afraid of getting "tangled up" because, as a long, thin, and flexible food item, it can easily become knotted or entwined with other objects - or in this case, a partner.\n\nThe humor comes from the double meaning of the phrase. In relationships, "tangled up" typically means to become deeply emotionally involved or complicated, but for spaghetti, it\'s a literal concern due to its physical properties. The joke relies on this wordplay to create a unexpected and silly connection between the setup (spaghetti refusing to get married) and the punchline (afraid of getting tangled up).\n\nThe joke requires a quick mental shift to understand the dual 

In [17]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.', 'explanation': 'A clever play on words. This joke is funny because it uses a common phrase associated with romantic relationships, "tangled up," and gives it a literal twist. In this case, the spaghetti is afraid of getting "tangled up" because, as a long, thin, and flexible food item, it can easily become knotted or entwined with other objects - or in this case, a partner.\n\nThe humor comes from the double meaning of the phrase. In relationships, "tangled up" typically means to become deeply emotionally involved or complicated, but for spaghetti, it\'s a literal concern due to its physical properties. The joke relies on this wordplay to create a unexpected and silly connection between the setup (spaghetti refusing to get married) and the punchline (afraid of getting tangled up).\n\nThe joke requires a quick mental shift to u

In [19]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.', 'explanation': 'A clever play on words. This joke is funny because it uses a common phrase associated with romantic relationships, "tangled up," and gives it a literal twist. In this case, the spaghetti is afraid of getting "tangled up" because, as a long, thin, and flexible food item, it can easily become knotted or entwined with other objects - or in this case, a partner.\n\nThe humor comes from the double meaning of the phrase. In relationships, "tangled up" typically means to become deeply emotionally involved or complicated, but for spaghetti, it\'s a literal concern due to its physical properties. The joke relies on this wordplay to create a unexpected and silly connection between the setup (spaghetti refusing to get married) and the punchline (afraid of getting tangled up).\n\nThe joke requires a quick mental shift to 

Time Travel

In [20]:

workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f0e0a57-6154-65f9-8000-62c056615864"}})

StateSnapshot(values={'topic': 'pasta'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f0e0a57-6154-65f9-8000-62c056615864'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-12-24T08:49:30.381669+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0a57-6151-6ef6-bfff-0a4daae31830'}}, tasks=(PregelTask(id='6ce54337-0c85-8628-ca3e-00c6d4eb26bd', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.'}),), interrupts=())

In [21]:
workflow.invoke(None,{"configurable": {"thread_id": "2", "checkpoint_id": "1f0e0a57-6154-65f9-8000-62c056615864"}})

{'topic': 'pasta',
 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.',
 'explanation': 'A clever play on words. This joke relies on a pun to create humor. Here\'s a breakdown of how it works:\n\nThe setup for the joke is "Why did the spaghetti refuse to get married?" which primes the listener to expect a reason related to relationships or commitment.\n\nThe punchline "Because it was afraid of getting tangled up in a relationship" is where the wordplay happens. "Tangled up" has a double meaning here:\n\n1. **Literal meaning**: Spaghetti is a type of long, thin, twisted pasta that can easily become tangled or knotted.\n2. **Figurative meaning**: In relationships, "tangled up" is an idiomatic expression that means to become deeply involved or entangled in a complicated situation, often in a way that\'s difficult to escape or resolve.\n\nThe joke exploits this dual meaning, using the physical properties of spaghetti (it

In [22]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.', 'explanation': 'A clever play on words. This joke relies on a pun to create humor. Here\'s a breakdown of how it works:\n\nThe setup for the joke is "Why did the spaghetti refuse to get married?" which primes the listener to expect a reason related to relationships or commitment.\n\nThe punchline "Because it was afraid of getting tangled up in a relationship" is where the wordplay happens. "Tangled up" has a double meaning here:\n\n1. **Literal meaning**: Spaghetti is a type of long, thin, twisted pasta that can easily become tangled or knotted.\n2. **Figurative meaning**: In relationships, "tangled up" is an idiomatic expression that means to become deeply involved or entangled in a complicated situation, often in a way that\'s difficult to escape or resolve.\n\nThe joke exploits this dual meaning, using the physical propert

update state

In [24]:
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f0e0a57-6154-65f9-8000-62c056615864", "checkpoint_ns": ""}},{'topic':'samosa'})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0e0a85-7e19-653a-8001-197badbd54a4'}}

In [25]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0a85-7e19-653a-8001-197badbd54a4'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-12-24T09:10:08.201452+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0e0a57-6154-65f9-8000-62c056615864'}}, tasks=(PregelTask(id='a19f3008-7847-4862-ed7f-fac90a36addf', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti refuse to get married?\n\nBecause it was afraid of getting tangled up in a relationship.', 'explanation': 'A clever play on words. This joke relies on a pun to create humor. Here\'s a breakdown of how it works:\n\nThe setup for the joke is "Why did the spaghetti refuse to get married?" which primes the list

In [26]:
workflow.invoke(None,{"configurable":{"thread_id":"2","checkpoint_id":"1f0e0a85-7e19-653a-8001-197badbd54a4"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crunchy" on the outside and "empty" on the inside.',
 'explanation': 'A clever joke. This joke is a play on words, using the physical characteristics of a samosa (a type of fried or baked pastry) to make a humorous comment on emotional well-being.\n\nThe punchline "it was feeling a little \'crunchy\' on the outside and \'empty\' on the inside" is a clever double entendre. In a literal sense, a samosa is typically crunchy on the outside (due to its fried or baked exterior) and empty on the inside (as it is often filled with a hollow space or a small amount of filling).\n\nHowever, the joke also uses these physical characteristics to make a metaphorical comment on the samosa\'s emotional state. "Feeling crunchy on the outside" can be interpreted as feeling tough, hardened, or defensive, while "feeling empty on the inside" suggests a sense of emotional hollowness, emptiness, or lack of fulf

In [27]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?\n\nBecause it was feeling a little "crunchy" on the outside and "empty" on the inside.', 'explanation': 'A clever joke. This joke is a play on words, using the physical characteristics of a samosa (a type of fried or baked pastry) to make a humorous comment on emotional well-being.\n\nThe punchline "it was feeling a little \'crunchy\' on the outside and \'empty\' on the inside" is a clever double entendre. In a literal sense, a samosa is typically crunchy on the outside (due to its fried or baked exterior) and empty on the inside (as it is often filled with a hollow space or a small amount of filling).\n\nHowever, the joke also uses these physical characteristics to make a metaphorical comment on the samosa\'s emotional state. "Feeling crunchy on the outside" can be interpreted as feeling tough, hardened, or defensive, while "feeling empty on the inside" suggests a sense of emotional hollowness, emptin